 # SpyGRT NewCalibration Example 

 **Author: Youssef Ben Bouchta<br>
 Last Edited: 17/03/2022**

 Aligning two cameras with real-time stream.

In [1]:
# Example of how to initialise stream objects and basic stram control commands.
# Author: Youssef Ben Bouchta
# Last Edited: 29/03/2022

#Imports
import pyrealsense2 as rs2
from spygrt.calibration import Calibrator
from spygrt.stream import Camera

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


## Initialisation of the calibrators object

The first step to using SpyGRT to calibrate the depth cameras is to initialise a **Calibrator** object for each camera. 

To do so we must:


1. Set up a real-sense context, which will allow us to access each of the RealSense devices that are currently connected. 


2. Initialise a **Camera** object for each RealSense device and a **Calibrator** object for each **Camera**. Note that **Calibrator** objects can also be initialised with a **Recording** type object as well. 

In [3]:
# Initialisation of calibrator objects using real-time streams. Look at how the 
# StreamAccess example for how to use a substitute a Recording object instead.

## Set up a realsense context to access connected devices
ctx = rs2.context()

## Initialising container for calibrator object
calibs = []

## For every realsense device connected, initialise a calibrator object
for dev in ctx.query_devices():
    calibs.append(Calibrator(Camera(dev)))


no maximum distance setting for camera model: Intel RealSense D415
no minimum distance setting for camera model: Intel RealSense D415
no maximum distance setting for camera model: Intel RealSense D415
no minimum distance setting for camera model: Intel RealSense D415


## Aligning the cameras to each other. 

The first step to aligning the camera system to the frame of reference defined by the calibration board is to align the cameras to one another. 

This is done by calling `align_cameras()` method of one of a source **Calibrator** object and giving the target **Calibrator** object as input. This will change the frame of reference of the source **Calibrator** to that of the target **Calibrator**

In this example: 

1. `calibs[0]` is the 'source' **Calibrator**
2. `calibs[1]` is the 'target' **Calibrator**. 

In [4]:
# Using the calibrator objects to obtain the calibration to the board frame of reference.

## Align the frame of reference of the two cameras.
calibs[0].align_cameras(calibs[1])

## Aligning the cameras to the board

Once the cameras are calibrated to one another, we align them to an external frame of reference defined by the calibration board. 


This is done by calling the `align_to_board()` method of one of the **Calibrator** object. This will change the frame of reference of the **Calibrator** oject to the one defined by the board. 

In the following code, we give the **Calibrator** object associated with the second cameras as an input argument. This applies the same transformation to the inputted **Calibrator**, thereby transforming its frame of reference to that of the board **IF** it the two **Calibrator** objects are already aligned. 

This is best practice as it minimises the calibration errors, but it is not necessary. We could instead call the `align_to_board()` method of each **Calibrator** object with input arguments. 


NB: This resulting frame of reference will match IEC coordinates if the board is set so that the middle is at isocenter, the LR laser is aligned to the rows and the SI laser is aligned with the columns. 

In [5]:
## Align the camera frame of reference to the board frame of reference.
calibs[1].align_to_board(cal2 = calibs[0])

## Saving the calibration to a .txt file.

Once the calibration is over, the result of the calibration is saved in the **T** parameter of each calibrator. 

We accomplish this by calling the `.write_cal()` method of each **Calibrator**, which writes the 4x4 calibration matrix into a file named *XXX_cal.txt* where XXX is the camera's serial number.

NB: The **T** matrix represents the transformation matrix from the camera's frame of reference to the lab's. It is the inverse of the extrinsic matrix which represents the transformation matrix from the lab's frame of reference to the camera's. 

In [6]:
# Outputs calibration to a text file.
for cal in calibs:
	cal.write_cal()
    
print(calibs[0].T)
print(calibs[1].T)

[[-0.904530 0.274793 -0.347389 0.290613],
 [0.0651338 0.863837 0.513721 -0.295628],
 [0.438119 0.438908 -0.793585 0.430724],
 [0.0 0.0 0.0 1.0]]
Tensor[shape={4, 4}, stride={4, 1}, Float32, CPU:0, 0x23626d43470]
[[-0.950654 -0.136896 0.282094 -0.19515],
 [0.0241335 0.865971 0.501571 -0.277487],
 [-0.312626 0.483131 -0.819091 0.448065],
 [0.0 0.0 0.0 1.0]]
Tensor[shape={4, 4}, stride={4, 1}, Float32, CPU:0, 0x23626d43560]
